In [1]:
import os
import inspect

from itertools import combinations

from learning_dist_metrics.ldm import LDM
from learning_dist_metrics.datasets import load_sample_data
from learning_dist_metrics.dist_metrics import *

In [2]:
samp_df = load_sample_data()
ldm = LDM(is_debug=True)
ldm.fit(samp_df.data, samp_df.sim_pairs)
#print inspect.getsource(ldm._fit)

Examples of S: [(3, 5), (3, 8), (3, 10), (3, 11), (3, 14)] 2450
Examples of D: [(0, 3), (0, 5), (0, 8), (0, 10), (0, 11)] 2500
Examples of X: [[  4.83648891  -5.705894    10.74258782]
 [  5.67309594  -5.07527604   4.30693957]
 [  3.52601767  -4.40324289   6.40668539]
 [ -6.8748598   -0.85057327  -1.39915847]
 [  4.85684614  -4.45084898   1.80052416]] (100, 3)
--- 5.64063978195 seconds ---


In [4]:
#print inspect.getsource(ldm._fit)
print ldm.get_transform_matrix()
#print inspect.getsource(ldm._fit)

[1.0, 0.0, 0.0]


In [12]:
from itertools import combinations
from scipy.optimize import minimize
from numpy import log

def get_exclusive_pairs(target_pairs, reference_pairs):
    """ Remove from target_paris the item (pairs) which
        has matches in reference_pairs.
        
        Parameters:
        -----------
        target_pairs: {list}, [(1, 2), (1, 3), ...]
        reference_pairs: {list}, [(2, 1), (10, 11)]

        Returns:  
        -------
    """
    res_pairs = target_pairs[:]
    for ref_pair in reference_pairs:
        for tgt_pair in res_pairs:
            if set(ref_pair) == set(tgt_pair):
                res_pairs.remove(tgt_pair)
                break
    return res_pairs

def get_unique_items(x_pairs, y_pairs):
    """Return all item mentioned either by x_pairs
       or y_pairs.
    """
    x_pairs.extend(y_pairs)
    res = []
    for a, b in x_pairs:
        if not a in res:
            res.append(a)
        if not b in res:
            res.append(b)
    return res


## ####################### ##
## Debug
## ####################### ##
X = samp_df.data
S = samp_df.sim_pairs
D = None

n_sample, n_features = X.shape

bnds = [(0, None)] * n_features # boundaries
init = [1] * n_features # initial weights

if D is None:
    all_pairs = [i for i in combinations(range(n_sample), 2)]
    D = get_exclusive_pairs(all_pairs, S)
else:
    covered_items = get_unique_items(S, D)
    X = np.delete(X, covered_items, 0)
                  
def objective_func(w):
    a = squared_sum_grouped_dist(S, X, w) * 1.0
    b = squared_sum_grouped_dist(D, X, w) * 1.0
    b = sum_grouped_dist(D, X, w) * 1.0
    return a - log(b)

print "Examples of S: %s" % S[:5], len(S)
print "Examples of D: %s" % D[:5], len(D)
print "Examples of X: %s" % X[:5, :], X.shape

Examples of S: [(3, 5), (3, 8), (3, 10), (3, 11), (3, 14)] 2450
Examples of D: [(0, 3), (0, 5), (0, 8), (0, 10), (0, 11)] 2500
Examples of X: [[  4.83648891  -5.705894    10.74258782]
 [  5.67309594  -5.07527604   4.30693957]
 [  3.52601767  -4.40324289   6.40668539]
 [ -6.8748598   -0.85057327  -1.39915847]
 [  4.85684614  -4.45084898   1.80052416]] (100, 3)


In [13]:
print bnds
print init

[(0, None), (0, None), (0, None)]
[1, 1, 1]


In [14]:
fitted = minimize(objective_func, init, method="L-BFGS-B", bounds = bnds)

ValueError: array must not contain infs or NaNs

In [67]:
print inspect.getsource(get_exclusive_pairs)

print " ***************** "

print inspect.getsource(get_unique_items)

def get_exclusive_pairs(target_pairs, reference_pairs):
    """ Remove from target_paris the item (pairs) which
        has matches in reference_pairs.
        
        Parameters:
        -----------
        target_pairs: {list}, [(1, 2), (1, 3), ...]
        reference_pairs: {list}, [(2, 1), (10, 11)]

        Returns:  
        -------
    """
    for ref_pair in reference_pairs:
        for tgt_pair in target_pairs:
            if set(ref_pair) == set(tgt_pair):
                target_pairs.remove(tgt_pair)
                break
    return target_pairs

 ***************** 
def get_unique_items(x_pairs, y_pairs):
    """Return all item mentioned either by x_pairs
       or y_pairs.
    """
    x_pairs.extend(y_pairs)
    res = []
    for a, b in x_pairs:
        if not a in res:
            res.append(a)
        if not b in res:
            res.append(b)
    return res



In [19]:
print objective_func(w = [.5, .5, .5])
print objective_func(w = [.5, .5, 0])
#a = squared_sum_grouped_dist(S, X, [1, 1, 1]) * 1.0

10032.0953349
3230.42865414


In [114]:
n_sample, _ = samp_df.data.shape
S = samp_df.sim_pairs
all_pairs = [i for i in combinations(range(n_sample), 2)]
D = get_exclusive_pairs(all_pairs, S)
print "n_sample: %d" % n_sample
print "all_pairs: %s" % all_pairs[:5], len(all_pairs)
print "S: %s" % S[:5], len(S)
print "D: %s" % D[:5], len(D)

n_sample: 100
all_pairs: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)] 4950
S: [(3, 5), (3, 8), (3, 10), (3, 11), (3, 14)] 24950
D: [] 0


In [119]:
items = []
for a, b in S:
    if not (a in items):
        items.append(a)
    if not (b in items):
        items.append(b)

In [122]:
items = sorted(items)
print items

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [98]:
x = [1, 2, 3]

In [104]:
l = x.copy()

AttributeError: 'list' object has no attribute 'copy'

In [110]:
def get_exclusive_pairs(target_pairs, reference_pairs):
    """ Remove from target_paris the item (pairs) which
        has matches in reference_pairs.
        
        Parameters:
        -----------
        target_pairs: {list}, [(1, 2), (1, 3), ...]
        reference_pairs: {list}, [(2, 1), (10, 11)]

        Returns:  
        -------
    """
    res_pairs = target_pairs[:]
    for ref_pair in reference_pairs:
        for tgt_pair in res_pairs:
            if set(ref_pair) == set(tgt_pair):
                res_pairs.remove(tgt_pair)
                break
    return res_pairs

In [112]:
get_exclusive_pairs([(1, 2), (2, 3), (2, 4)], [(1, 2)])

[(2, 3), (2, 4)]

In [ ]:
import pdb as pdb

def functionWithBreakpoint():
    number = 0
    for i in range(10):
        number += 2
        pdb.set_trace()

        numnber -= 1
        pdb.set_trace()

    return number

print functionWithBreakpoint()